# 数据读取 - PyTorch 的 Dataset 初探

CSDN 基本是憨批写的半截教程，看了半天发现写的都是垃圾半吊子东西，还是要我自己写教程。

在 TF 中，它是 `keras.utils.Sequence`，在 PyTorch 中，它是 `torch.utils.data.Dataset`，目的都是通过迭代器的形式来读取大数据集。

参考资料：[How to use dataset larger than memory?
](https://discuss.pytorch.org/t/how-to-use-dataset-larger-than-memory/37785)

## Dataset 总览

首先看 `torch.utils.data.Dataset`这个抽象类。可以使用这个抽象类来构造 PyTorch 数据集。

要注意的是以这个类构造的子类，一般要定义两个函数：

- `__getitem__`：必须定义， supporting fetching a data sample for a given key，根据一个 key 去取一个 data。；

- `__len__`：可选定义，返回数据集的大小（size）；

首先我们定义一个简单的数据集。

In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler

In [2]:
class NewDataSet(Dataset):
    """这是一个Dataset子类。"""
    def __init__(self, csv_file):
        """初始，读取数据"""
        self.csv_data = pd.read_csv(csv_file)
 
    def __getitem__(self, index):
        """按 index，每次读取一条数据"""
        data = self.csv_data.values[index]
        return data
 
    def __len__(self):
        """返回 csv_data 的总长度"""
        return len(self.csv_data)

我们创建了一个 `TxtDataset` 对象，并调用函数。

注意 `__getitem__` 的调用要通过 `对象[索引]` 调用。

In [3]:
# 创建对象
iris = NewDataSet("Iris.csv")

print(iris.__len__())

150


In [4]:
print(iris)

In [5]:
print(iris[0])

[1 5.1 3.5 1.4 0.2 'Iris-setosa']


## DataSet 类构建

### 整个读取

实际上我们的 DataLoader 是不能读取 string 类型的 label 的，而 iris 数据集的 label 恰好就是 string 类型的数据。

因此我们要重写 `NewDataSet` 类，将数据清洗部分也放入其中。

首先我们先用传统方法来读取文件。

In [16]:
df_iris = pd.read_csv("Iris.csv", encoding='utf-8')

In [17]:
df_iris.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [18]:
df_iris

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [19]:
set(df_iris['Species'].values)

{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}

对于 `Iris.csv` 文件，要把最后一列“Species”改为数字类型，并且我们并不需要 `Id` 这一列。

通常做法如下：

In [20]:
le = LabelEncoder()
df_iris['Species'] = le.fit_transform(df_iris['Species'])  # 处理种类
df_iris = df_iris.drop(['Id'], axis=1)                     # 删除Id

In [21]:
df_iris.head(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Chunksize

我们的终极目标是按批次读取 `Iris.csv` 文件，首先我们熟悉一下 Chunksize 读取功能。

> 当然 `Iris.csv` 的数据量很小，只有区区 150 条。
>
> 但是想象一下，如果有一个很大很大的 CSV 文件（或者其他文件、数据库表单等），以至于直接将其读为 DataFrame 格式保存在内存中很困难怎么办？
>
> 有两种办法：
>
> 一、我们就需要利用 `pd.read_csv()` 的 `chunksize` 功能一批一批地读取。
>
> 二、我们利用 `pd.read_csv(iterator=True)` 和 `get_chunk(size)` 配合读取。
>
> 两种原理差不多，都是以迭代器的形式处理的。


以 `pd.read_csv()` 为例，参考 [IO tools](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) 和 [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) 文档，具体代码如下：

In [22]:
df_iris_batch = pd.read_csv("Iris.csv",         # 文件名 
                            encoding='utf-8',   # 解码
                            # header=None,        # 不读取标题
                            iterator=True       # 开启迭代功能
                           )

In [24]:
print(df_iris_batch)
print("\n")
print(type(df_iris_batch))
print("\n")
print(df_iris_batch.get_chunk(20))



<class 'pandas.io.parsers.TextFileReader'>


    Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
7    8            5.0           3.4            1.5           0.2  Iris-setosa
8    9            4.4           2.9            1.4           0.2  Iris-setosa
9   10            4.9           3.1            1.5           0.1  Iris-setosa
10  11            5.4           3.7            1.5           0.2  Iris-setosa
11  12            4.8           3.4            1.6           0.2  Iris-setosa
12  13            4.8           3.0            1.4           0.1  Iris-setosa
13  14            4.3           3.0            1.1           0.1  Iris-setosa
14  15            5.8           4.0            1.2           0.2  Iris-setosa
15  16            5.7           4.4            1.5           0.4  Iris-setosa
16  17            5.4           3.9            1.3           0.4  Iris-setosa
17  18            5.1           3.5            1.4           0.3  Iris-setosa
18  19           

我们可以看出，使用了 `chunksize` 功能后，df_iris_batch 已经不是一个 `DataFrame` 了，而是一个 `TextFileReader`，或者通俗地说，变成了一个 `iterator`。

我们可以使用 `for` 循环来读取：

In [88]:
class NewDataSet(Dataset):
    """这是一个Dataset子类。"""
    def __init__(self, csv_file, chunkSize, n_samples):
        """初始，读取数据"""
        self.chunksize = chunkSize
        self.reader = pd.read_csv(csv_file,       # 文件名 
                            encoding='utf-8',     # 解码 
                            iterator=True
                           )
        self.len = math.ceil(n_samples / self.chunksize)
 
    def __getitem__(self, index):
        """每次读取一个 chunk_size 的数据，包括预处理"""
        data = self.reader.get_chunk(self.chunksize)
        # 替换
        data = data.replace({'Species': {'Iris-setosa': 0, 
                                         'Iris-versicolor': 1,
                                         'Iris-virginica': 2}})
        data['Species'] = data['Species'].astype(int)
        # 转换为 Tensor
        tensorData = torch.tensor(data.values, dtype=torch.float32)
        inputs = tensorData[:, 1:5]  # 特征
        labels = tensorData[:, 5]    # 标签
        return inputs, labels 
 
    def __len__(self):
        """返回总的长度，也就是按照 chunk_size 的迭代次数"""
        return self.len

In [90]:
# 创建对象
iris = NewDataSet("Iris.csv", chunkSize=7, n_samples=150)

print(iris.__len__())

21


最后看看我们的 `Dataset` 的结果。

> 需要注意的是，如果在迭代完成以后再次调用 `next()` 方法，会抛出 `StopIteration` 异常。

In [91]:
print(iris[0])

(tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000],
        [5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.4000, 1.4000, 0.3000]]), tensor([0., 0., 0., 0., 0., 0., 0.]))


## 接入 DataLoader

在 PyTorch 中，我们使用 `torch.utils.data.DataLoader` 从打包好的 `DataSet` 中读取数据。

如果我们使用原来的 `TensorDataSet` 是很简单的，由于这里自定义了 `NewDataSet` 类，所以也来试试。

### Sampler

首先使用 `RandomSampler` 对数据进行采样。

RandomSampler 待补充。

In [103]:
iris = NewDataSet("Iris.csv", 
                  chunkSize=10,    # chunkSize
                  n_samples=150)  # 总数据量
print(iris.__len__())

# sampler = RandomSampler(iris) 

15


### DataLoader

In [104]:
dataloader = DataLoader(iris, 
                        batch_size=10, 
                        num_workers=1, 
                        shuffle=False)

In [105]:
for step, batch in enumerate(dataloader):
    print(f"当前step: {step}")
    train_X, train_y = batch
    print(f"当前特征内容：{train_X}")

当前step: 0
当前特征内容：tensor([[[5.1000, 3.5000, 1.4000, 0.2000],
         [4.9000, 3.0000, 1.4000, 0.2000],
         [4.7000, 3.2000, 1.3000, 0.2000],
         [4.6000, 3.1000, 1.5000, 0.2000],
         [5.0000, 3.6000, 1.4000, 0.2000],
         [5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.4000, 1.4000, 0.3000],
         [5.0000, 3.4000, 1.5000, 0.2000],
         [4.4000, 2.9000, 1.4000, 0.2000],
         [4.9000, 3.1000, 1.5000, 0.1000]],

        [[5.4000, 3.7000, 1.5000, 0.2000],
         [4.8000, 3.4000, 1.6000, 0.2000],
         [4.8000, 3.0000, 1.4000, 0.1000],
         [4.3000, 3.0000, 1.1000, 0.1000],
         [5.8000, 4.0000, 1.2000, 0.2000],
         [5.7000, 4.4000, 1.5000, 0.4000],
         [5.4000, 3.9000, 1.3000, 0.4000],
         [5.1000, 3.5000, 1.4000, 0.3000],
         [5.7000, 3.8000, 1.7000, 0.3000],
         [5.1000, 3.8000, 1.5000, 0.3000]],

        [[5.4000, 3.4000, 1.7000, 0.2000],
         [5.1000, 3.7000, 1.5000, 0.4000],
         [4.6000, 3.6000, 1.0000,